# Spotify listening history

In [1]:
sample(c("Andreas","Robin","Markus"))

[1] "Andreas" "Robin"   "Markus"

In [2]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──



✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1



── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Data loading

In [3]:
data <- tibble(js=jsonlite::read_json("spotify.json")) %>% unnest_wider(js)

In [4]:
glimpse(data)

Rows: 103,945
Columns: 17
$ ts                                <chr> "2016-12-11T19:08:05Z", "2021-04-18T…
$ ms_played                         <int> 117, 87270, 209200, 157026, 172630, …
$ conn_country                      <chr> "DE", "DE", "DE", "DE", "DE", "DE", …
$ master_metadata_track_name        <chr> "Bless Us All", "I Love You (But I H…
$ master_metadata_album_artist_name <chr> "Tiny Tim", "Neon Trees", "Moop Mama…
$ master_metadata_album_album_name  <chr> "The Muppets Christmas Carol", "Pop …
$ spotify_track_uri                 <chr> "spotify:track:7LaoCUikJDSPNMpqGs6Aw…
$ episode_name                      <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ episode_show_name                 <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ spotify_episode_uri               <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ reason_start                      <chr> "trackdone", "fwdbtn", "trackdone", …
$ reason_end                        <chr> "endplay", "fwdbtn", "trackdone", "t…
$ shuffle     

In [5]:
head(data)

ts,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<dbl>,<lgl>
2016-12-11T19:08:05Z,117,DE,Bless Us All,Tiny Tim,The Muppets Christmas Carol,spotify:track:7LaoCUikJDSPNMpqGs6Awr,NA,NA,NA,trackdone,endplay,FALSE,NA,FALSE,0.000000e+00,FALSE
2021-04-18T15:10:17Z,87270,DE,I Love You (But I Hate Your Friends),Neon Trees,Pop Psychology,spotify:track:3DuTGtayotEsUNokov5ZS4,NA,NA,NA,fwdbtn,fwdbtn,TRUE,NA,FALSE,1.618759e+12,FALSE
2021-05-07T13:27:02Z,209200,DE,Liebe,Moop Mama,Das Rote Album,spotify:track:6TWn9w7vcFLMnau9sBkH4z,NA,NA,NA,trackdone,trackdone,TRUE,NA,FALSE,1.620394e+12,FALSE
2017-03-29T19:11:55Z,157026,DE,The Imitation Game,Alexandre Desplat,The Imitation Game (Original Motion Picture Soundtrack),spotify:track:0o9ivTBX7mjTnaUYF4Gk6t,NA,NA,NA,clickrow,trackdone,TRUE,NA,FALSE,1.490815e+12,FALSE
2021-04-28T10:46:24Z,172630,DE,Boys,Lizzo,Boys,spotify:track:3R9RSIkgkZXrIgmBHGemlV,NA,NA,NA,trackdone,trackdone,FALSE,NA,FALSE,1.619607e+12,FALSE
2015-10-03T19:37:55Z,260713,DE,Madness,Muse,The 2nd Law,spotify:track:0c4IEciLCDdXEhhKxj4ThA,NA,NA,NA,trackdone,endplay,FALSE,TRUE,FALSE,0.000000e+00,FALSE


## Favorite Music

### Get the overall top song

In [6]:
data %>% count(master_metadata_track_name) %>% arrange(desc(n)) %>% head

master_metadata_track_name,n
<chr>,<int>
NA,2540
Bitch,244
Boys,156
"Sunrise Mass: The Spheres, ""Kyrie eleison""",147
Herz an Herz,145
Tota pulchra es,141


In [7]:
data %>% filter(!is.na(master_metadata_track_name)) %>% count(master_metadata_track_name, master_metadata_album_artist_name) %>% arrange(desc(n)) %>% head(10)

master_metadata_track_name,master_metadata_album_artist_name,n
<chr>,<chr>,<int>
Bitch,Von Wegen Lisbeth,177
"Sunrise Mass: The Spheres, ""Kyrie eleison""",Ola Gjeilo,147
Tota pulchra es,Ola Gjeilo,141
Lay All Your Love On Me,ABBA,139
Karambolage,Panda Lux,135
Herz an Herz,Blümchen,132
Immaterial,SOPHIE,130
Denkmal,Wir Sind Helden,122
Nur ein Wort,Wir Sind Helden,120


#### Exclude songs that didn't play for more than 30 seconds

In [8]:
data %>% filter(!is.na(master_metadata_track_name), ms_played > 30000) %>% count(master_metadata_track_name, master_metadata_album_artist_name) %>% arrange(desc(n)) %>% head(10)

master_metadata_track_name,master_metadata_album_artist_name,n
<chr>,<chr>,<int>
Bitch,Von Wegen Lisbeth,126
Karambolage,Panda Lux,117
Immaterial,SOPHIE,103
Herz an Herz,Blümchen,102
Denkmal,Wir Sind Helden,99
Lay All Your Love On Me,ABBA,97
Wieso,Von Wegen Lisbeth,97
Nur ein Wort,Wir Sind Helden,93
deine cousine,MELE,92


#### Most skipped songs: listened to for less than 10 seconds

In [9]:
data %>% filter(!is.na(master_metadata_track_name), ms_played < 10000) %>% count(master_metadata_track_name, master_metadata_album_artist_name) %>% arrange(desc(n)) %>% head(10)

master_metadata_track_name,master_metadata_album_artist_name,n
<chr>,<chr>,<int>
"Sunrise Mass: The Spheres, ""Kyrie eleison""",Ola Gjeilo,139
Tota pulchra es,Ola Gjeilo,138
It's Beginning to Look a Lot like Christmas,Michael Bublé,52
恋は永遠,GING NANG BOYZ,43
Hammer Horror,Kate Bush,42
Bitch,Von Wegen Lisbeth,38
Cake By The Ocean,DNCE,37
Green Light,Lorde,37
Lay All Your Love On Me,ABBA,36


In [10]:
data %>%
    filter(!is.na(master_metadata_track_name), ms_played < 10000) %>% 
    group_by(master_metadata_track_name, master_metadata_album_artist_name) %>% 
    summarize(n=n(),mean_playtime = mean(ms_played)) %>% 
    filter(mean_playtime > 0) %>%
    arrange((mean_playtime)) %>%
    head(10)

`summarise()` has grouped output by 'master_metadata_track_name'. You can
override using the `.groups` argument.


master_metadata_track_name,master_metadata_album_artist_name,n,mean_playtime
<chr>,<chr>,<int>,<dbl>
恋は永遠,GING NANG BOYZ,43,1.232558
Excavation,Sjellos,1,2.000000
LA PA PARADISE,BRADIO,10,3.000000
Storm on Boat (Insomnia Treatment),Natural Sounds Music Academy,1,3.000000
"Symphony No. 8 in E flat - ""Symphony of a Thousand"" / Part Two: Final scene from Goethe's ""Faust"": ""Bei der Liebe"" - ""Bei dem Bronn"" - Bei dem hochgeweihten Orte""",Gustav Mahler,1,3.000000
Let Me Go,Sunrise Avenue,10,4.200000
Hellwalker,Mick Gordon,1,5.000000
Cloud Atlas End Title,Tom Tykwer,1,8.000000
Ole Ole,Yizhar Kohen,1,8.000000
